#### lalalalala (deletar)

1. J002- Nas duas últimas semanas, ___ deixou de realizar quaisquer de suas atividades habituais (trabalhar, ir à escola, brincar, afazeres domésticos etc.) por motivo da própria saúde
2. J00402- Qual foi o principal motivo de saúde que impediu ___ de realizar suas atividades habituais nas duas últimas semanas
3. J00404- Este motivo de saúde estava relacionado ao trabalho

se J002 = 2, então 0.

se J002 = 1 e J00402 = 8 e J00404 = 1, então 1.

se J002 = 1 e J00402 = 8 e J00404 = 2, então 2.


In [5]:
condicoes = [
    df['J002'] == 2,
    (df['J002'] == 1) & (df['J00402'] == 8) & (df['J00404'] == 1),
    (df['J002'] == 1) & (df['J00402'] == 8) & (df['J00404'] == 2)
]

# Resultados correspondentes às condições
valores = [0, 1, 2]

# Crie o novo atributo
df['AusenciaTrabalhoPorDC'] = np.select(condicoes, valores, default=np.nan)

In [6]:
print(df['AusenciaTrabalhoPorDC'].isna().sum())


2639


In [7]:
df['AusenciaTrabalhoPorDC'] = df['AusenciaTrabalhoPorDC'].fillna(0)

In [8]:
df = df.drop(columns=['J001', 'J002', 'J00402', 'J00404'])

#### Produto cartesiano N010, N011, N012 e N016

In [90]:
import itertools

# Supondo que essas sejam as categorias possíveis (ajuste conforme o dicionário da PNS)
categorias = [1, 2, 3, 4]  # Por exemplo: 1=Nunca, 2=Raramente, ..., 5=Sempre

# Produto cartesiano
combinacoes = list(itertools.product(categorias, repeat=4))

# Transformar em DataFrame para visualização
import pandas as pd
df_combinacoes = pd.DataFrame(combinacoes, columns=['N010', 'N011', 'N012', 'N016'])

print(df_combinacoes.head())


   N010  N011  N012  N016
0     1     1     1     1
1     1     1     1     2
2     1     1     1     3
3     1     1     1     4
4     1     1     2     1


In [91]:
# Como string: para criar um código tipo "1-2-3"
df['sono_combinado'] = df['N010'].astype(str) + '-' + df['N011'].astype(str) + '-' + df['N012'].astype(str)

# Ou como tupla (útil para agrupar e contar depois)
df['sono_combinado_tuple'] = list(zip(df['N010'], df['N011'], df['N012']))


In [92]:
# Como DataFrame
frequencias = df['sono_combinado'].value_counts().reset_index()
frequencias.columns = ['sono_combinado', 'frequencia']

frequencias


,sono_combinado,frequencia
0,1.0-1.0-1.0,10577
1,2.0-1.0-1.0,1610
2,1.0-2.0-1.0,1309
3,2.0-2.0-2.0,1074
4,2.0-2.0-1.0,922
...,...,...
59,3.0-4.0-3.0,19
60,2.0-1.0-4.0,17
61,2.0-3.0-4.0,16
62,3.0-1.0-4.0,10


In [93]:
df = df.drop(columns=['sono_combinado', 'sono_combinado_tuple'])

#### Produto cartesiano P04501 e P04502

In [96]:
import itertools
import pandas as pd

# Categorias possíveis: opções de 1 a 6
categorias = list(range(1, 7))

# Produto cartesiano
combinacoes = list(itertools.product(categorias, categorias))

# Transformar em DataFrame
df_combinacoes = pd.DataFrame(combinacoes, columns=['P04501', 'P04502'])

print(df_combinacoes)

    P04501  P04502
0        1       1
1        1       2
2        1       3
3        1       4
4        1       5
5        1       6
6        2       1
7        2       2
8        2       3
9        2       4
10       2       5
11       2       6
12       3       1
13       3       2
14       3       3
15       3       4
16       3       5
17       3       6
18       4       1
19       4       2
20       4       3
21       4       4
22       4       5
23       4       6
24       5       1
25       5       2
26       5       3
27       5       4
28       5       5
29       5       6
30       6       1
31       6       2
32       6       3
33       6       4
34       6       5
35       6       6


In [175]:
# calculo do imc, mas eu provavelmente vou mudar ele de lugar, pois precisamos inputar primeiro

import numpy as np

# Calculando o IMC (peso / altura²) e convertendo altura de cm para metros
df['imc'] = df['P00104'] / ((df['P00404'] / 100) ** 2)

# Classificando em faixas
conditions = [
    (df['imc'] < 18.5),
    ((df['imc'] >= 18.5) & (df['imc'] < 25)),
    ((df['imc'] >= 25) & (df['imc'] < 30)),
    ((df['imc'] >= 30) & (df['imc'] < 35)),
    ((df['imc'] >= 35) & (df['imc'] < 40)),
    (df['imc'] >= 40)
]

labels = [
    'Abaixo do peso',
    'Peso normal',
    'Sobrepeso',
    'Obesidade Grau I',
    'Obesidade Grau II',
    'Obesidade Grau III'
]

# Criando a coluna com as faixas
df['faixa_imc'] = np.select(conditions, labels, default='Dados inválidos')

# Opcional: Arredondar o IMC para 2 casas decimais
df['imc'] = df['imc'].round(2)

In [176]:
df['imc']

0        25.88
1        34.05
2        36.89
3        28.38
4        28.96
         ...  
24340    24.84
24341    23.62
24342    22.32
24343    25.51
24344    31.53
Name: imc, Length: 24345, dtype: float64